In [1]:
import numpy as np
import mido
import pretty_midi
import sklearn
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 600

In [2]:
def convert_histograms_into_notes(hist_series, time_difference=1):
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(0)
    midi.instruments = [instrument]
    
    
    
    instrument.notes = []
    counter = 0
    for hist in hist_series:
        pitch = 21
        for note in hist:
            note *= 127
            note = int(note // 1)
            midiNote = pretty_midi.Note(note, pitch, counter, counter+time_difference)
            instrument.notes.append(midiNote)
            
            pitch += 1
            
        counter += time_difference
    
    return midi

In [24]:
class Agent:
    def __init__(self):
        self.pos = np.random.randint(50,76)
        self.remain_at_same = 0
        self.allowed_moves = np.random.randint(0,8,3)
        
    def update(self, hist):
        #interval_weights = [0.6,0.2,0]
        
        desired_index = -1
        if self.remain_at_same > 5:
            least_weight = 2
            for i in [-12, -9, -8, -7, -5, -4, -3, 3, 4, 5, 7, 8, 9, 12]:
                i += self.pos - 21
                if i >= 0 and i < len(hist):
                    weight = hist[i]
                    if weight < least_weight:
                        desired_index = i
                        least_weight = weight
        elif np.random.random() < 0.1:
            index = self.pos - 21

            desired_index = -1
            best_weight = -1

            for i in range(index-7, index+8):
                if i >= 0 and i < len(hist):
                    #print(i, len(hist), index)
                    weight = hist[i]
                    if weight > best_weight:
                        desired_index = i
                        best_weight = weight
        
        if self.pos == desired_index + 21:
            self.remain_at_same += 1
        else:
            self.remain_at_same = 0
        
            move_to_make = np.random.choice(self.allowed_moves)
            if desired_index + 21 > self.pos:
                self.pos += move_to_make
                if self.pos > 108:
                    self.pos = 108
            else:
                self.pos -= move_to_make
                if self.pos < 21:
                    self.pos = 21

In [7]:
class System:
    def __init__(self, number_of_agents):
        
        self.agents = [Agent() for i in range(number_of_agents)]
        self.make_hist()
    
    def make_hist(self):
        histogram = np.zeros(109-21)
        for agent in self.agents:
            index = agent.pos - 21
            histogram[index] += 1
        
        histogram = histogram/np.max(histogram)
        
        self.hist = histogram
    
    def update(self):
        for agent in self.agents:
            agent.update(self.hist)
        
        self.make_hist()
        
        return self.hist

In [25]:
sy = System(2000)

histogram = [sy.hist]
for i in range(100):
    histogram.append(sy.update())

midi = convert_histograms_into_notes(histogram, 0.1)

midi.write("PitchVelocityExperiment.mid")

In [19]:
print(histogram[0])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.91578947
 0.81052632 0.76842105 0.84210526 0.81052632 0.81052632 0.76842105
 0.78947368 0.87368421 0.70526316 0.78947368 0.81052632 0.82105263
 0.81052632 0.73684211 0.97894737 1.         1.         0.92631579
 0.77894737 0.8        0.69473684 0.84210526 0.57894737 0.58947368
 0.8        0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Fixing random state for reproducibility
np.random.seed(19680801)
# Fixing bin edges
HIST_BINS = np.linspace(21, 109, 100)

# histogram our data with numpy
n = histogram[0]

def prepare_animation(bar_container):

    def animate(frame_number):
        n = histogram[frame_number]
        for count, rect in zip(n, bar_container.patches):
            rect.set_height(count)
        return bar_container.patches
    return animate

fig, ax = plt.subplots()
_, _, bar_container = ax.hist(data, HIST_BINS, lw=1,
                              ec="yellow", fc="green", alpha=0.5)
ax.set_ylim(top=1)  # set safe limit to ensure that all data is visible.

ani = animation.FuncAnimation(fig, prepare_animation(bar_container), 50,
                              repeat=False, blit=True)
ani.save("histogram.gif")

MovieWriter ffmpeg unavailable; using Pillow instead.
